# Inheritance: For Good orFor Worse

```
started to push on the inheritance idea as a way to let novices build on frameworks that could only be designed by expert.
```

## 多重继承和方法解析顺序

![](https://hotttao.gitbooks.io/fluent-python-note/image/inherit_order.png)

> Python3 MRO是广度优先遍历。

In [1]:
class A:
    def ping(self):
        print('ping', self)

        
class B(A):
    def pong(self):
        print('pong', self)

        
class C(A):
    def pong(self):
        print('pong', self)

        
class D(B, C):
    def ping(self):
        super().ping()
        print('post-ping', self)
    
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)

In [2]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [3]:
D().pingpong()

ping <__main__.D object at 0x112db6f90>
post-ping <__main__.D object at 0x112db6f90>
ping <__main__.D object at 0x112db6f90>
pong <__main__.D object at 0x112db6f90>
pong <__main__.D object at 0x112db6f90>
pong <__main__.D object at 0x112db6f90>


## 处理多重继承

Coping with Multiple Inheritance

```
we need to a better theory about inheritance entirely. For example, inheritance and instancing (which is a kind of inheritance) muddles both pragmatics (such as factoring code to save space) and semantics (used for way too many tasks such as: specialization, generalization, speciation, etc.)

--Alan Kay
The Early History of Smalltalk
```

### 把接口继承和实现区分开

Distinguish Interface Inheritance from Implementation Inheritance

使用多重继承时，一定要明确一开为什么创建子类。主要原因可能有：

* 继承接口，创建子类型，实现“是什么”关系
* 继承实现，通过重用避免代码重复

通过继承重用代码是实现细节，通常可以换用组合和委托模式。而接口继承则是框架的支柱。

```
Inheritance for code reuse is an implementation detail, and it can often be replaced by composition and delegation. On the other hand, interface inheritance is the backbone of a framework.
```

### 使用抽象基类显示表示接口

如果类的作用是定义接口，应该明确把它定义为抽象基类。

```
If a class is designed an interface, it should be an explicit ABC.
```

### 通过混入重用代码

Use Mixins for Code Reuse.

**如果一个类的作用是为多个不相关的子类提供方法实现，从而实现重用，但不体现“是什么”关系，应该把哪个类明确地定义为混入类(mixin class)。**从概念上讲，混入不定义新类型，只是打包方法，以便重用。混入类决定不能实例化，而且具体类不能即成混入类。混入类应该是提供某方面的特定行为，只实现少量关系非常紧密的方法。

### 在名称中明确指明混入

因为在Python没有把类声明为混入的正规方法，所以强烈建议在名称加入...Mixin后缀。

### 抽象类可以作为混入，反过来则不成立

抽象基类可以实现具体方法，因此也可以作为混入使用。不过，抽象基类会定义类型，而混入做不到。此外，抽象基类可以作为其他类的唯一基类，而混入绝不能作为唯一的超类。

### 不要子类化多个具体类

Don't Subclass from More Than One Concrete Class.

```
Concrete classes should have zero or at most one concrete superclass. In other words, all but one of the superclasses of a concrete class should be ABCs.
```

### 为用户提供聚合类

Provide Aggregate Class to Users.

如果抽象基类或混入的组合对客户代码非常有用，那就提供一个类，使用易于理解的方式把他们结合起来。Grady Booch把这种类成为聚合类(aggregate class)。

### 优先使用对象组合，而不是类继承

## Django通用视图的混入

![](https://hotttao.gitbooks.io/fluent-python-note/image/inherit_django.png)

View是所有的视图的基类，提供核心功能，如dispatch方法。这个方法委托具体子类实现的处理方法(handler)，如get, head, post等，处理不同的HTTP请求。

View具体的子类是处理方法。

TemplateResponseMixin提供的功能只针对需要使用模版的视图。